https://www.kaggle.com/competitions/playground-series-s4e4

Private Score: 0.14613.

Public score: 0.14633.

Main references:

1. Abalone https://archive.ics.uci.edu/dataset/1/abalone,
2. How to Transform Target Variables for Regression in Python https://machinelearningmastery.com/how-to-transform-target-variables-for-regression-with-scikit-learn/,
3. Comparison of Regression Analysis Algorithms https://gursev-pirge.medium.com/comparison-of-regression-analysis-algorithms-db710b6d7528,
4. https://github.com/szilard/benchm-ml,
5. EDA from PlaygroundS4E04|EDA|Baseline https://www.kaggle.com/code/ravi20076/playgrounds4e04-eda-baseline,
6. Ensemble from https://www.kaggle.com/competitions/playground-series-s4e4/discussion/492815#2773797,
7. Feature engineering from PS4E4 | Abalone Age Prediction | Regression by MINATO NAMIKAZE https://www.kaggle.com/code/arunklenin/ps4e4-abalone-age-prediction-regression,
8. optuna parameters from PS4E4: XGB + Cat + LGB + HGB https://www.kaggle.com/code/martinapreusse/ps4e4-xgb-cat-lgb-hgb#Feature-Engineering.

Finally,
9. PS4E4 🏆| XGBoost+LIGHTGBM+CatBoost😊😊😊 https://www.kaggle.com/code/aaachen/ps4e4-xgboost-lightgbm-catboost.

Sex: M, F, and I (infant),

Length: Longest shell measurement (mm),

Diameter: perpendicular to length (mm),

Height: with meat in shell (mm),

Whole_weight: whole abalone (grams),

Shucked_weight: weight of meat (grams),

Viscera_weight: gut weight (after bleeding) (grams),

Shell_weight: after being dried (grams),

Rings: +1.5 gives the age in years.

In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from warnings import filterwarnings;
filterwarnings('ignore');

from scipy import stats
from scipy.stats import norm, skew, boxcox_normmax
from scipy.special import boxcox1p

from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.ensemble import StackingRegressor, VotingRegressor
from sklearn.compose import TransformedTargetRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import optuna

from datetime import datetime

# 1. Load Data

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s4e4/train.csv')
train.rename(columns={'Whole weight':'Whole_weight', 'Whole weight.1':'Shucked_weight', 'Whole weight.2':'Viscera_weight', 'Shell weight':'Shell_weight'}, inplace=True)
train.drop('id', axis=1, inplace=True)

In [3]:
original = pd.read_csv('/kaggle/input/abalone-extra/abalone.data', sep=',', names=['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight', 'Rings'])

In [4]:
test = pd.read_csv('/kaggle/input/playground-series-s4e4/test.csv')
test.rename(columns={'Whole weight':'Whole_weight', 'Whole weight.1':'Shucked_weight', 'Whole weight.2':'Viscera_weight', 'Shell weight':'Shell_weight'}, inplace=True)
test_ID = test['id']
test.drop('id', axis=1, inplace=True)

# 2. EDA

In [ ]:
def MakeCatFtrePlots(cat_cols):
        "This method returns the category feature plots";
        
        fig, axes = plt.subplots(len(cat_cols), 3, figsize = (20, len(cat_cols)* 4.5), 
                                     gridspec_kw = {'wspace': 0.25, 'hspace': 0.3});

        for i, col in enumerate(cat_cols):
            ax = axes[i, 0] if len(cat_cols) > 1 else axes[0];
            a = train[col].value_counts(normalize = True);
            a.sort_index().plot.barh(ax = ax, color = '#007399');
            ax.set_title(f"{col}_Train", **title_specs);
            ax.set_xticks(np.arange(0.0, 0.9, 0.05), 
                              labels = np.round(np.arange(0.0, 0.9, 0.05),2), 
                              rotation = 90
                             );
            ax.set(xlabel = '', ylabel = '');
            del a;

            ax = axes[i, 1] if len(cat_cols) > 1 else axes[1];
            a = test[col].value_counts(normalize = True);
            a.sort_index().plot.barh(ax = ax, color = '#0088cc');
            ax.set_title(f"{col}_Test", **title_specs);
            ax.set_xticks(np.arange(0.0, 0.9, 0.05), 
                              labels = np.round(np.arange(0.0, 0.9, 0.05),2), 
                              rotation = 90
                             );
            ax.set(xlabel = '', ylabel = '');
            del a;

            ax = axes[i, 2] if len(cat_cols) > 1 else axes[2];
            a = original[col].value_counts(normalize = True);
            a.sort_index().plot.barh(ax = ax, color = '#0047b3');
            ax.set_title(f"{col}_Original", {'fontsize': 9, 'fontweight': 'bold', 'color': '#992600'});
            ax.set_xticks(np.arange(0.0, 0.9, 0.05), 
                              labels = np.round(np.arange(0.0, 0.9, 0.05),2), 
                              rotation = 90
                             );
            ax.set(xlabel = '', ylabel = '');
            del a;       

        plt.suptitle(f"Category column plots", **title_specs, y= 0.975);
        plt.tight_layout();
        plt.show();

In [ ]:
def MakeContColPlots(cont_cols):
        "This method returns the continuous feature plots";
        
        df = pd.concat([train[cont_cols].assign(Source = 'Train'), 
                            test[cont_cols].assign(Source = 'Test'),
                            original[cont_cols].assign(Source = "Original")
                           ], 
                           axis=0, ignore_index = True
                          );

        fig, axes = plt.subplots(len(cont_cols), 4 ,figsize = (16, len(cont_cols) * 4.2), 
                                     gridspec_kw = {'hspace': 0.35, 
                                                    'wspace': 0.3, 
                                                    'width_ratios': [0.80, 0.20, 0.20, 0.20]
                                                   }
                                    );

        for i,col in enumerate(cont_cols):
            ax = axes[i,0];
            sns.kdeplot(data = df[[col, 'Source']], x = col, hue = 'Source', 
                            palette = ['#0039e6', '#ff5500', '#00b300'], 
                            ax = ax, linewidth = 2.1
                           );
            ax.set_title(f"\n{col}", **title_specs);
            ax.grid(grid_specs);
            ax.set(xlabel = '', ylabel = '');

            ax = axes[i,1];
            sns.boxplot(data = df.loc[df.Source == 'Train', [col]], y = col, width = 0.25,
                            color = '#33ccff', saturation = 0.90, linewidth = 0.90, 
                            fliersize= 2.25,
                            ax = ax);
            ax.set(xlabel = '', ylabel = '');
            ax.set_title(f"Train", **title_specs);

            ax = axes[i,2];
            sns.boxplot(data = df.loc[df.Source == 'Test', [col]], y = col, width = 0.25, fliersize= 2.25,
                            color = '#80ffff', saturation = 0.6, linewidth = 0.90, 
                            ax = ax); 
            ax.set(xlabel = '', ylabel = '');
            ax.set_title(f"Test", **title_specs);

            ax = axes[i,3];
            sns.boxplot(data = df.loc[df.Source == 'Original', [col]], y = col, width = 0.25, fliersize= 2.25,
                            color = '#99ddff', saturation = 0.6, linewidth = 0.90, 
                            ax = ax); 
            ax.set(xlabel = '', ylabel = '');
            ax.set_title(f"Original", **title_specs);

        plt.suptitle(f"\nDistribution analysis- continuous columns\n", **title_specs,
                         y = 0.92, x = 0.50
                        );
        plt.tight_layout();
        plt.show();

In [ ]:
title_specs = {'fontsize': 9, 'fontweight': 'bold', 'color': '#992600'};
grid_specs = {'visible': True, 'which': 'both', 'linestyle': '--', 'color': 'lightgrey', 'linewidth': 0.75};

In [ ]:
MakeCatFtrePlots(['Sex'])

In [ ]:
MakeContColPlots(['Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight', 'Shell_weight'])

Because both the train and test datasets show similar distribution patterns for all the feature variables, while the original dataset different, we only consider the train dataset before inferring on test dataset.

In [5]:
train = pd.concat([train, original], axis=0)

In [6]:
train.drop_duplicates(inplace=True)

In [7]:
# No missing values in the training dataset.
train.isna().sum().sum()

0

In [ ]:
def plot_hist_distributions(df, numcols=3, fig_kwargs = dict()):
    fig, axes = plt.subplots(1+len(df.columns)//numcols, numcols, **fig_kwargs)
    for col, ax in zip(df.columns, axes.flatten()):
        sns.histplot(df[col], ax=ax, bins=30)
        #ax.set_title(col, size=10)
    plt.tight_layout()

In [ ]:
plot_hist_distributions(train, fig_kwargs={'figsize':[12,8]})

skewness < 0: Length and Diameter,
skewness > 0: Whole_weight, Shucked_weight, Viscera_weight, Shell_weight and Rings,
skewness ~0 : Height.

# 3. Removing Outliers

In [ ]:
def plot_scatter_distributions(df, numcols=3, fig_kwargs = dict()):
    fig, axes = plt.subplots(1+len(df.columns)//numcols, numcols, **fig_kwargs)
    for col, ax in zip(df.columns, axes.flatten()):
        sns.scatterplot(df, x=col, y='Rings', ax=ax)
        #ax.set_title(col, size=10)
    plt.tight_layout()

In [ ]:
train_scatter = train.drop(['Sex'], axis=1)
plot_scatter_distributions(train_scatter, fig_kwargs={'figsize':[12,8]})

In [8]:
train.drop(train[(train['Rings']==29)&(train['Length']<0.4)].index, inplace=True)
train.drop(train[train['Height'] > 0.4].index, inplace=True)
# The same as train[(train['Rings']<5)&(train['Shell_weight'] > 0.4)].index
train.drop(train[(train['Rings']<5)&(train['Whole_weight']>1.5)].index, inplace=True)
train.drop(train[train['Viscera_weight'] > 0.7].index, inplace=True)

# 4. Add new features.

In [ ]:
def new_features(data):
    df = data.copy()
    
    # Clean the weights by capping the over weights with total body weights
    df['Shell_weight'] = np.where(df['Shell_weight'] > df['Whole_weight'], df['Whole_weight'], df['Shell_weight'])
    df['Viscera_weight'] = np.where(df['Viscera_weight'] > df['Whole_weight'], df['Whole_weight'], df['Viscera_weight'])
    df['Shucked_weight'] = np.where(df['Shucked_weight'] > df['Whole_weight'], df['Whole_weight'], df['Shucked_weight'])

    # Abalone density approx
    df['Approx_density'] = df['Whole_weight'] / (df['Length'] * df['Diameter'] * df['Height'] + 1e-5)

    # Abalone BMI
    df['BMI'] = df['Whole_weight'] / (df['Height']**2+1e-5)

    # Measurement derived
    df['D_L_ratio'] = df['Diameter'] / (df['Length'] + 1e-5)
    df['H_L_ratio'] = df['Height'] / (df['Length'] + 1e-5)
    df['Shell_Shuck_ratio'] = df["Shell_weight"] / (df["Shucked_weight"] + 1e-5)
    df['Shell_Viscera_ratio'] = df['Shell_weight'] / (df['Viscera_weight'] + 1e-5)

    df['Viscera_Whole_ratio'] = df['Viscera_weight'] / (df['Whole_weight']  +1e-5)
    df['Shell_Whole_ratio'] = df['Shell_weight'] / (df['Whole_weight']    +1e-5)
    df['Shuck_Whole_ratio'] = df['Shucked_weight'] / (df['Whole_weight']   +1e-5)

    # Water Loss during experiment
    df["water_loss"] = df["Whole_weight"] - df["Shucked_weight"] - df['Viscera_weight'] - df['Shell_weight']
    df["water_loss"] = np.where(df["water_loss"]<0, min(df["Shucked_weight"].min(), df["Viscera_weight"].min(), df["Shell_weight"].min()), df["water_loss"])
    
    df['Med_Height'] = df.groupby('Sex')['Height'].transform('median')
    df['Med_Shell_weight'] = df.groupby('Sex')['Shell_weight'].transform('median')
    
    return df

In [ ]:
#train=new_features(train)
#test=new_features(test)

# 5. Normalize the target variable.

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
#train["Rings"] = np.log1p(train["Rings"])

In [ ]:
#Check the new distribution 
#sns.histplot(train["Rings"], kde=True, stat="density", kde_kws=dict(cut=3), alpha=.4, edgecolor=(1, 1, 1, .4), bins=30);

# Get the fitted parameters used by the function
#(mu, sigma) = norm.fit(train["Rings"])
#print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

# Plot the PDF. 
#xmin, xmax = plt.xlim() 
#x = np.linspace(xmin, xmax, 100) 
#p = norm.pdf(x, mu, sigma) 
  
#plt.plot(x, p, 'k', linewidth=2) 
#plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)], loc='best')
#plt.title('Rings distribution')

#Get also the QQ-plot
#fig = plt.figure()
#res = stats.probplot(train["Rings"], plot=plt)
#plt.show()

# 6. Normalize all numerical independent variables.

In [9]:
X_train = train.drop('Rings', axis=1)
y_train = train['Rings']

In [10]:
numeric_features = X_train.dtypes[X_train.dtypes != "object"].index
print('\n Skewness in all numerical independent variables: \n')
skewness_info = pd.DataFrame({'Skewness': X_train[numeric_features].apply(lambda x: skew(x))})
skewness_info.head(30)


 Skewness in all numerical independent variables: 



,Skewness
Length,-0.727601
Diameter,-0.691166
Height,-0.270921
Whole_weight,0.434817
Shucked_weight,0.599726
Viscera_weight,0.481997
Shell_weight,0.488141


In [11]:
df = skewness_info[abs(skewness_info['Skewness']) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(df.shape[0]))

There are 0 skewed numerical features to Box Cox transform


In [ ]:
# Because there are 0 values in the Height column, we can not use stats.boxcox.
#skewed_features = df.index
#for i in skewed_features:
#    X_train[i] = boxcox1p(X_train[i], boxcox_normmax(X_train[i] + 1, method='mle'))

In [ ]:
# Distributions tend to be normal.
#plot_hist_distributions(X_train, fig_kwargs={'figsize':[12,8]})
#plot_scatter_distributions(pd.concat([X_train.drop(['Sex'], axis=1), y_train], axis=1), fig_kwargs={'figsize':[12,8]})

In [12]:
X_train = pd.get_dummies(X_train, columns=['Sex'], dtype=float)

In [ ]:
#numeric_cols = pd.concat([X_train, y_train], axis=1).select_dtypes(include='number')
#corrmat = numeric_cols.corr()
#plt.subplots(figsize=(15,15))
#sns.heatmap(corrmat, vmax=0.9, square=True, annot=True)

# 7. Compare Models

Because RandomForestRegressor runs slowly, we choose the other four tree-based models.

In [13]:
#cv = KFold(n_splits=10, shuffle=True, random_state=42)

#def score_cv(model):
#    return -cross_val_score(model, X_train, y_train, scoring="neg_root_mean_squared_error", cv=cv)

# We define a rmsle evaluation function
def evaluation_metric(y, y_pred):
    rmsle = mean_squared_log_error(y, y_pred, squared=False)
    return rmsle

In [ ]:
#model_randomforest = RandomForestRegressor()
#score = score_cv(model_randomforest)
#print("\n RandomForestRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

#model_randomforest.fit(X_train, y_train)
#train_pred_randomforest = model_randomforest.predict(X_train)
#print(evaluation_metric(y_train, train_pred_randomforest))

model_ridge = make_pipeline(RobustScaler(), RidgeCV(cv=cv))
score = score_cv(model_ridge)
print("\n Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

model_ridge.fit(X_train, y_train)
train_pred_ridge = model_ridge.predict(X_train)
print(evaluation_metric(y_train, train_pred_ridge))

model_histgradientboost = HistGradientBoostingRegressor()
score = score_cv(model_histgradientboost)
print("\n HistGradientBoostingRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

model_histgradientboost.fit(X_train, y_train)
train_pred_histgradientboost = model_histgradientboost.predict(X_train)
print(evaluation_metric(y_train, train_pred_histgradientboost))

model_lightgbm = LGBMRegressor(force_col_wise=True, verbose=-1)
score = score_cv(model_lightgbm)
print("\n LGBMRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

model_lightgbm.fit(X_train, y_train)
train_pred_lightgbm = model_lightgbm.predict(X_train)
print(evaluation_metric(y_train, train_pred_lightgbm))

model_xgboost = XGBRegressor()
score = score_cv(model_xgboost)
print("\n XGBRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

model_xgboost.fit(X_train, y_train)
train_pred_xgboost = model_xgboost.predict(X_train)
print(evaluation_metric(y_train, train_pred_xgboost))

model_catboost = CatBoostRegressor(verbose=False)
score = score_cv(model_catboost)
print("\n CatBoostRegressor score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

model_catboost.fit(X_train, y_train)
train_pred_catboost = model_catboost.predict(X_train)
print(evaluation_metric(y_train, train_pred_catboost))

In [ ]:
#level0 = list()
#level0.append(('ridge', model_ridge))
#level0.append(('hgb', model_histgradientboost))
#level0.append(('lgbm', model_lightgbm))
#level0.append(('xgb', model_xgboost))
#level0.append(('cb', model_catboost))
# define meta learner model
#level1 = model_catboost
#model_stacking = StackingRegressor(estimators=level0, final_estimator=level1, cv=5)

#score = score_cv(model_stacking)
#print("\n StackingRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

#model_stacking.fit(X_train, y_train)
#train_pred_stacking = model_stacking.predict(X_train)
#print(evaluation_metric(y_train, train_pred_stacking))

model_voting = VotingRegressor([('hgb', model_histgradientboost), ('lgbm', model_lightgbm), ('xgb', model_xgboost), ('cb', model_catboost)])
score = score_cv(model_voting)
print("\n VotingRegressor score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

model_voting.fit(X_train, y_train)
train_pred_voting = model_voting.predict(X_train)
print(evaluation_metric(y_train, train_pred_voting))

evaluation metric on train dataset
train_pred =  train_pred_voting
print(evaluation_metric(y_train, train_pred))

# 8. Fine-tune models

We will use Optuna to fine-tune four models: model_histgradientboost, model_lightgbm, model_xgboost and model_catboost.

In [14]:
# stratify parameter keeps the ratio of Rings same all across the Dataset
X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)
X_val, X_te, y_val, y_te = train_test_split(X_te, y_te, test_size=0.5, random_state=42, stratify=y_te)

# # 8.1 XGBoost

In [15]:
def xgb_objective(trial):
    params = {
        "eta": trial.suggest_float("eta", 0.01, 1.0),
        "gamma": 0.0,
        "max_depth": trial.suggest_int("max_depth", 3, 20),
        "min_child_weight": trial.suggest_float("min_child_weight", 1., 50.),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_lambda": trial.suggest_float("lambda", 1.0, 100.0),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000)
    }
    
    xgb_reg = TransformedTargetRegressor(XGBRegressor(**params, objective='reg:squarederror', grow_policy='lossguide',
                                                         tree_method="hist", random_state=42),
                                                         func=np.log1p,
                                                         inverse_func=np.expm1)
    
    xgb_reg.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    
    val_scores = evaluation_metric(y_val, xgb_reg.predict(X_val))
    return val_scores

sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization
xgb_study = optuna.create_study(direction = 'minimize',study_name="XgbRegressor", sampler=sampler)

[I 2024-04-30 12:19:21,648] A new study created in memory with name: XgbRegressor


In [16]:
# Set TUNE parameter to True in case you want to run Hyper Parameter Tuning
TUNE = False
if TUNE:
    xgb_study.optimize(xgb_objective, n_trials=1000)

XGBoost 1

In [17]:
xgb_best_params_1 = {
    'eta': 0.010005505674323728,
    'max_depth': 10,
    'min_child_weight': 12.87582166258377,
    'subsample': 0.8415240730082273,
    'colsample_bytree': 0.6803730288580919,
    'lambda': 8.565844294512017,
    'n_estimators': 982
}

In [18]:
xgb_reg_1 = TransformedTargetRegressor(XGBRegressor(**xgb_best_params_1, objective='reg:squarederror', grow_policy='lossguide',
                                                 tree_method="hist", random_state=42, gamma=0.0),
                                                         func=np.log1p,
                                                         inverse_func=np.expm1)

In [19]:
xgb_reg_1.fit(X_tr, y_tr)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=XGBRegressor(base_score=None, booster=None,
                                                  callbacks=None,
                                                  colsample_bylevel=None,
                                                  colsample_bynode=None,
                                                  colsample_bytree=0.6803730288580919,
                                                  device=None,
                                                  early_stopping_rounds=None,
                                                  enable_categorical=False,
                                                  eta=0.010005505674323728,
                                                  eval_metric=None,
                                                  feature_types=None, ga...
                                                  grow_policy='lossguide',
                                                  importance_type=None,
                                                  interaction_constraints=None,
                                                  lambda=8.565844294512017,
                                                  learning_rate=None,
                                                  max_bin=None,
                                                  max_cat_threshold=None,
                                                  max_cat_to_onehot=None,
                                                  max_delta_step=None,
                                                  max_depth=10, max_leaves=None,
                                                  min_child_weight=12.87582166258377,
                                                  missing=nan,
                                                  monotone_constraints=None,
                                                  multi_strategy=None,
                                                  n_estimators=982, n_jobs=None, ...))

In [20]:
evaluation_metric(y_val, xgb_reg_1.predict(X_val))

0.1478477436837093

In [ ]:
feature_importance = xgb_reg_1.regressor_.feature_importances_
feature_names = X_tr.columns

sorted_indices = feature_importance.argsort()
sorted_importance = feature_importance[sorted_indices]
sorted_features = feature_names[sorted_indices]

plt.figure(figsize=(10, 6))
colors = plt.cm.tab20c.colors[:len(sorted_features)]  
plt.barh(sorted_features, sorted_importance, color=colors)
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('XGBoost Feature Importance')
plt.gca().invert_yaxis() 
plt.tight_layout()  
plt.show()

XGBoost 2

In [21]:
xgb_best_params_2 = {
    'eta': 0.01077242260892917,
    'max_depth': 11,
    'min_child_weight': 9.114122932027431,
    'subsample': 0.8209497225389015,
    'colsample_bytree': 0.5537485302252583,
    'lambda': 10.201959897562874,
    'n_estimators': 949 
}

In [22]:
xgb_reg_2 = TransformedTargetRegressor(XGBRegressor(**xgb_best_params_2, objective='reg:squarederror', grow_policy='lossguide',
                                                 tree_method="hist", random_state=42, gamma=0.0),
                                                         func=np.log1p,
                                                         inverse_func=np.expm1)

In [23]:
xgb_reg_2.fit(X_tr, y_tr)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=XGBRegressor(base_score=None, booster=None,
                                                  callbacks=None,
                                                  colsample_bylevel=None,
                                                  colsample_bynode=None,
                                                  colsample_bytree=0.5537485302252583,
                                                  device=None,
                                                  early_stopping_rounds=None,
                                                  enable_categorical=False,
                                                  eta=0.01077242260892917,
                                                  eval_metric=None,
                                                  feature_types=None, gam...
                                                  grow_policy='lossguide',
                                                  importance_type=None,
                                                  interaction_constraints=None,
                                                  lambda=10.201959897562874,
                                                  learning_rate=None,
                                                  max_bin=None,
                                                  max_cat_threshold=None,
                                                  max_cat_to_onehot=None,
                                                  max_delta_step=None,
                                                  max_depth=11, max_leaves=None,
                                                  min_child_weight=9.114122932027431,
                                                  missing=nan,
                                                  monotone_constraints=None,
                                                  multi_strategy=None,
                                                  n_estimators=949, n_jobs=None, ...))

In [24]:
evaluation_metric(y_val, xgb_reg_2.predict(X_val))

0.14778540063608597

# # 8.2 LightGBM

In [25]:
def lgbm_objective(trial):
    # Define parameters to be optimized for the LGBMClassifier
    param = {
        "verbosity": -1,
        "random_state": 42,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.05),
        "n_estimators": trial.suggest_int("n_estimators", 400, 1000),
        "lambda_l1": trial.suggest_float("lambda_l1", 0.005, 0.015),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.02, 0.06),
        "max_depth": trial.suggest_int("max_depth", 6, 14),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.9),
        "subsample": trial.suggest_float("subsample", 0.8, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 10, 70),
        "num_leaves": trial.suggest_int("num_leaves", 30, 100),
        "min_split_gain": trial.suggest_float("min_split_gain", 0.1, 1.0)
    }

    lgbm_reg = TransformedTargetRegressor(LGBMRegressor(**param),
                                                         func=np.log1p,
                                                         inverse_func=np.expm1)
    
    lgbm_reg.fit(X_tr, y_tr, eval_set=[(X_val, y_val)])

    val_scores = evaluation_metric(y_val, lgbm_reg.predict(X_val))
    return val_scores


# Set up the sampler for Optuna optimization
sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization

# Create a study object for Optuna optimization
lgbm_study = optuna.create_study(direction="minimize", study_name="LGBMRegressor", sampler=sampler)

[I 2024-04-30 12:20:30,531] A new study created in memory with name: LGBMRegressor


In [28]:
TUNE = False
if TUNE:
    # Run the optimization process
    lgbm_study.optimize(lambda trial: lgbm_objective(trial), n_trials=400)

    # Get the best parameters after optimization
    lgbm_best_params = lgbm_study.best_params

    print('='*50)
    print(lgbm_best_params)

Lightgbm 1

In [29]:
lgbm_params_1 = {
    'learning_rate': 0.023935877142742104, 
    'n_estimators': 427, 
    'lambda_l1': 0.014169833960239667, 
    'lambda_l2': 0.02453214295703588, 
    'max_depth': 12, 
    'colsample_bytree': 0.6339394447109402, 
    'subsample': 0.8878090464357417, 
    'min_child_samples': 18, 
    'num_leaves': 94, 
    'min_split_gain': 0.10176601252417374,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}

In [30]:
lgbm_reg_1 = TransformedTargetRegressor(LGBMRegressor(**lgbm_params_1),
                                                 func=np.log1p,
                                                 inverse_func=np.expm1)

In [31]:
lgbm_reg_1.fit(X_tr, y_tr)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=LGBMRegressor(colsample_bytree=0.6339394447109402,
                                                   lambda_l1=0.014169833960239667,
                                                   lambda_l2=0.02453214295703588,
                                                   learning_rate=0.023935877142742104,
                                                   max_depth=12,
                                                   min_child_samples=18,
                                                   min_split_gain=0.10176601252417374,
                                                   n_estimators=427, n_jobs=-1,
                                                   num_leaves=94,
                                                   random_state=42,
                                                   subsample=0.8878090464357417,
                                                   verbose=-1))

In [32]:
evaluation_metric(y_val, lgbm_reg_1.predict(X_val))

0.14861673532786357

In [ ]:
feature_importance = lgbm_reg_1.regressor_.feature_importances_

feature_names = X_tr.columns

sorted_indices = feature_importance.argsort()
sorted_importance = feature_importance[sorted_indices]
sorted_features = feature_names[sorted_indices]

# Plot feature importance
plt.figure(figsize=(12, 8))
colors = plt.cm.Paired.colors[:len(sorted_features)]  
plt.barh(sorted_features, sorted_importance, color=colors)
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('LightGBM Feature Importance', fontsize=14)
plt.gca().invert_yaxis() 

for i, v in enumerate(sorted_importance):
    plt.text(v + 0.02, i, f'{v:.2f}', color='black', va='center', fontsize=10)

plt.tight_layout()  
plt.show()

Lightgbm 2

In [33]:
lgbm_params_2 = {
    'learning_rate': 0.01711102441624917, 
    'n_estimators': 784, 
    'lambda_l1': 0.014968558836396594, 
    'lambda_l2': 0.024025038402942282, 
    'max_depth': 13, 
    'colsample_bytree': 0.6237421121741338, 
    'subsample': 0.8114831522568513, 
    'min_child_samples': 12, 
    'num_leaves': 92, 
    'min_split_gain': 0.10028969929666237,
    'random_state': 42,
    'n_jobs': -1,
    'verbose': -1
}

In [34]:
lgbm_reg_2 = TransformedTargetRegressor(LGBMRegressor(**lgbm_params_2),
                                                 func=np.log1p,
                                                 inverse_func=np.expm1)

In [35]:
lgbm_reg_2.fit(X_tr, y_tr)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=LGBMRegressor(colsample_bytree=0.6237421121741338,
                                                   lambda_l1=0.014968558836396594,
                                                   lambda_l2=0.024025038402942282,
                                                   learning_rate=0.01711102441624917,
                                                   max_depth=13,
                                                   min_child_samples=12,
                                                   min_split_gain=0.10028969929666237,
                                                   n_estimators=784, n_jobs=-1,
                                                   num_leaves=92,
                                                   random_state=42,
                                                   subsample=0.8114831522568513,
                                                   verbose=-1))

In [36]:
evaluation_metric(y_val, lgbm_reg_2.predict(X_val))

0.14857253688976363

# # 8.3 CatBoost

In [37]:
def cb_objective(trial):
    params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.5),
        "max_depth": trial.suggest_int("depth", 4, 16),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1500),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),
    }
    
    cb_reg = TransformedTargetRegressor(CatBoostRegressor(**params, random_state=42, grow_policy='SymmetricTree',
                               random_strength=0, loss_function="RMSE"),
                                                     func=np.log1p,
                                                     inverse_func=np.expm1)
    
    cb_reg.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
    
    val_scores = evaluation_metric(y_val, cb_reg.predict(X_val))
    return val_scores

sampler = optuna.samplers.TPESampler(seed=42)  # Using Tree-structured Parzen Estimator sampler for optimization
cb_study = optuna.create_study(direction = 'minimize',study_name="CBRegressor", sampler=sampler)

[I 2024-04-30 12:48:43,551] A new study created in memory with name: CBRegressor


In [40]:
TUNE = False
if TUNE:
    cb_study.optimize(cb_objective, 60)

Catboost 1

In [41]:
cb_params_1 = {
    'learning_rate': 0.10605445426625311,
    'depth': 6,
    'l2_leaf_reg': 9.206629836873022,
    'n_estimators': 1383,
    'colsample_bylevel': 0.6773475586070697,
    'grow_policy': 'SymmetricTree',
    'random_strength': 0, 
    'boost_from_average': True, 
    'loss_function': 'RMSE',
    'verbose': False
    }

In [42]:
cat_reg_1 = TransformedTargetRegressor(CatBoostRegressor(**cb_params_1),
                                                     func=np.log1p,
                                                     inverse_func=np.expm1)

In [43]:
cat_reg_1.fit(X_tr, y_tr)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=<catboost.core.CatBoostRegressor object at 0x7a87e0e42770>)

In [44]:
evaluation_metric(y_val, cat_reg_1.predict(X_val))

0.1479613952786055

Catboost 2

In [45]:
cb_params_2 = {
    'learning_rate': 0.11286727482639491,
    'depth': 5,
    'l2_leaf_reg': 9.140871295975039,
    'n_estimators': 1173,
    'colsample_bylevel': 0.6185578793283091,
    'grow_policy': 'SymmetricTree',
    'random_strength': 0, 
    'boost_from_average': True, 
    'loss_function': 'RMSE',
    'verbose': False
    }

In [46]:
cat_reg_2 = TransformedTargetRegressor(CatBoostRegressor(**cb_params_2),
                                                     func=np.log1p,
                                                     inverse_func=np.expm1)

In [47]:
cat_reg_2.fit(X_tr, y_tr)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=<catboost.core.CatBoostRegressor object at 0x7a87e0e6c6a0>)

In [48]:
evaluation_metric(y_val, cat_reg_2.predict(X_val))

0.14809846783148756

# # 8.4 HGBoost 

In [49]:
def hgb_objective(trial):
    params = {
        "max_leaf_nodes": trial.suggest_int('max_leaf_nodes', 10, 40),
        "learning_rate": trial.suggest_float('learning_rate', 0, 0.5),
        "max_depth": trial.suggest_int('max_depth', 3, 20),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 10, 40),
        "max_bins": trial.suggest_int('max_bins', 2, 255),
        "random_state": 42
    }
    
    hgb_reg = TransformedTargetRegressor(HistGradientBoostingRegressor(**params),
                                                     func=np.log1p,
                                                     inverse_func=np.expm1)
    
    # , eval_set=[(X_val, y_val)]
    hgb_reg.fit(X_tr, y_tr)
    
    val_scores = evaluation_metric(y_val, hgb_reg.predict(X_val))
    return val_scores

sampler = optuna.samplers.TPESampler(seed=42)
hgb_study = optuna.create_study(direction="minimize", study_name="HGBRegressor", sampler=sampler)

[I 2024-04-30 13:53:36,012] A new study created in memory with name: HGBRegressor


In [56]:
TUNE = False
if TUNE:
    # Start optimizing with 200 trials
    hgb_study.optimize(hgb_objective, 200)

HGBoost 1

In [52]:
hgb_params_1 = {
    'max_leaf_nodes': 37,
    'learning_rate': 0.1295378116724848,
    'max_depth': 20,
    'min_samples_leaf': 37,
    'max_bins': 237,
    'random_state': 42
}

In [53]:
hgb_reg_1 = TransformedTargetRegressor(HistGradientBoostingRegressor(**hgb_params_1),
                                                     func=np.log1p,
                                                     inverse_func=np.expm1)

In [54]:
hgb_reg_1.fit(X_tr, y_tr)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=HistGradientBoostingRegressor(learning_rate=0.1295378116724848,
                                                                   max_bins=237,
                                                                   max_depth=20,
                                                                   max_leaf_nodes=37,
                                                                   min_samples_leaf=37,
                                                                   random_state=42))

In [55]:
evaluation_metric(y_val, hgb_reg_1.predict(X_val))

0.1490761161309024

# # 8.5 RandomForest 

https://www.kaggle.com/code/mustafagerme/optimization-of-random-forest-model-using-optuna

In [57]:
# Define objective function
def rf_objective(trial):
    # Suggest values for hyperparameters
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 10, 200, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 32),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "n_jobs": -1, 
        "random_state": 42
    }
    
    
    rf_reg = TransformedTargetRegressor(RandomForestRegressor(**params),
                                                     func=np.log1p,
                                                     inverse_func=np.expm1)
    
    # , eval_set=[(X_val, y_val)]
    rf_reg.fit(X_tr, y_tr)
    
    val_scores = evaluation_metric(y_val, rf_reg.predict(X_val))
    return val_scores

sampler = optuna.samplers.TPESampler(seed=42)
rf_study = optuna.create_study(direction="minimize", study_name="RFRegressor", sampler=sampler)

[I 2024-04-30 14:03:43,120] A new study created in memory with name: RFRegressor


In [60]:
# Run optimization process
TUNE = False
if TUNE:
    rf_study.optimize(rf_objective, n_trials=50, show_progress_bar=True)

Random Forest 1

In [61]:
rf_params_1 = {
    'n_estimators': 195,
    'max_depth': 13,
    'min_samples_split': 10,
    'min_samples_leaf': 8,
    "n_jobs": -1, 
    "random_state": 42
}

In [62]:
rf_reg_1 = TransformedTargetRegressor(RandomForestRegressor(**rf_params_1),
                                                     func=np.log1p,
                                                     inverse_func=np.expm1)

In [63]:
rf_reg_1.fit(X_tr, y_tr)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=RandomForestRegressor(max_depth=13,
                                                           min_samples_leaf=8,
                                                           min_samples_split=10,
                                                           n_estimators=195,
                                                           n_jobs=-1,
                                                           random_state=42))

In [64]:
evaluation_metric(y_val, rf_reg_1.predict(X_val))

0.14947121053973828

# # 8.6 Voting

In [65]:
ensemble = VotingRegressor(
    [
        ("xgb_1", xgb_reg_1),
        ("xgb_2", xgb_reg_2),
#        ("lgbm_1", lgbm_reg_1),
#        ("lgbm_2", lgbm_reg_2),
        ("cb_1", cat_reg_1),
        ("cb_2", cat_reg_2),
#        ("hgb_1", hgb_reg_1)
    ]
)

In [66]:
ensemble.fit(X_train, y_train)

VotingRegressor(estimators=[('xgb_1',
                             TransformedTargetRegressor(func=<ufunc 'log1p'>,
                                                        inverse_func=<ufunc 'expm1'>,
                                                        regressor=XGBRegressor(base_score=None,
                                                                               booster=None,
                                                                               callbacks=None,
                                                                               colsample_bylevel=None,
                                                                               colsample_bynode=None,
                                                                               colsample_bytree=0.6803730288580919,
                                                                               device=None,
                                                                               early_stopping_rounds=None,
                                                                               enable_categorical=False,
                                                                               eta=0.010005505674323728,
                                                                               e...
                             TransformedTargetRegressor(func=<ufunc 'log1p'>,
                                                        inverse_func=<ufunc 'expm1'>,
                                                        regressor=<catboost.core.CatBoostRegressor object at 0x7a87e0e6c6a0>)),
                            ('hgb_1',
                             TransformedTargetRegressor(func=<ufunc 'log1p'>,
                                                        inverse_func=<ufunc 'expm1'>,
                                                        regressor=HistGradientBoostingRegressor(learning_rate=0.1295378116724848,
                                                                                                max_bins=237,
                                                                                                max_depth=20,
                                                                                                max_leaf_nodes=37,
                                                                                                min_samples_leaf=37,
                                                                                                random_state=42)))])

# 9. Make prediction on test dataset.

In [67]:
X_test = test

In [68]:
# No missing values in the test dataset.
X_test.isna().sum().sum()

0

In [69]:
numeric_features = X_test.dtypes[X_test.dtypes != "object"].index
print('\n Skewness in all numerical independent variables: \n')
skewness_info = pd.DataFrame({'Skewness': X_test[numeric_features].apply(lambda x: skew(x))})
skewness_info.head(30)


 Skewness in all numerical independent variables: 



,Skewness
Length,-0.734547
Diameter,-0.696294
Height,0.554492
Whole_weight,0.435653
Shucked_weight,0.593191
Viscera_weight,0.476117
Shell_weight,0.468512


In [70]:
df = skewness_info[abs(skewness_info['Skewness']) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(df.shape[0]))

There are 0 skewed numerical features to Box Cox transform


In [ ]:
# Because there are 0 values in the Height column, we can not use stats.boxcox.
#skewed_features = df.index
#for i in skewed_features:
#    X_test[i] = boxcox1p(X_test[i], boxcox_normmax(X_test[i] + 1, method='mle'))

In [71]:
X_test = pd.get_dummies(X_test, columns=['Sex'], dtype=float)

In [72]:
pred = ensemble.predict(X_test)

In [73]:
sub = pd.DataFrame()
sub['id'] = test_ID
sub['Rings'] = pred
sub['Rings'] = sub['Rings'].clip(1,29)
#sub['Rings'] = np.where(sub['Rings'].between(27.5, 29), 29, sub['Rings'])

In [74]:
sub.to_csv('/kaggle/working/submission.csv',index=False)
sub.head()

,id,Rings
0,90615,9.750440
1,90616,9.729701
2,90617,9.926200
3,90618,10.374029
4,90619,7.566330
